In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Function to split dataset
def split_dataset(original_dir, train_dir, val_dir, split_ratio=0.8):
    categories = os.listdir(original_dir)
    for category in categories:
        category_path = os.path.join(original_dir, category)
        if not os.path.isdir(category_path):
            continue
        
        images = os.listdir(category_path)
        train_images, val_images = train_test_split(images, test_size=1-split_ratio)
        
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(val_dir, category), exist_ok=True)
        
        for img in train_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_dir, category, img))
        
        for img in val_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(val_dir, category, img))

# Define paths
original_dataset = "dataset/original"
train_dir = "dataset/train"
val_dir = "dataset/validation"

# Create train and validation directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split the dataset
split_dataset(original_dataset, train_dir, val_dir)
print("Dataset split into training and validation sets.")


Dataset split into training and validation sets.


In [6]:
import os
from PIL import Image

# Directories for your dataset
directories = ['dataset/train', 'dataset/validation']

def validate_images(directory):
    for sub_dir in os.listdir(directory):
        sub_dir_path = os.path.join(directory, sub_dir)
        if not os.path.isdir(sub_dir_path):
            continue
        
        for file_name in os.listdir(sub_dir_path):
            file_path = os.path.join(sub_dir_path, file_name)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify the image integrity
            except Exception as e:
                print(f"Invalid image: {file_path} - {e}")
                os.remove(file_path)  # Remove the invalid file
                print(f"Removed corrupted file: {file_path}")

# Validate train and validation directories
for directory in directories:
    print(f"Validating images in {directory}...")
    validate_images(directory)


Validating images in dataset/train...
Invalid image: dataset/train\No_ID\good-ideas-young-attractive-student.jpg - cannot identify image file 'C:\\Users\\Phani\\Downloads\\dataset\\train\\No_ID\\good-ideas-young-attractive-student.jpg'
Removed corrupted file: dataset/train\No_ID\good-ideas-young-attractive-student.jpg
Invalid image: dataset/train\No_ID\happy-student-with-laptop.jpg - cannot identify image file 'C:\\Users\\Phani\\Downloads\\dataset\\train\\No_ID\\happy-student-with-laptop.jpg'
Removed corrupted file: dataset/train\No_ID\happy-student-with-laptop.jpg
Invalid image: dataset/train\No_ID\student-wearing-a-backpack-on-white-background.jpg - cannot identify image file 'C:\\Users\\Phani\\Downloads\\dataset\\train\\No_ID\\student-wearing-a-backpack-on-white-background.jpg'
Removed corrupted file: dataset/train\No_ID\student-wearing-a-backpack-on-white-background.jpg
Invalid image: dataset/train\Wearing_ID\confident-teacher-in-library.jpg - cannot identify image file 'C:\\Users\

In [8]:
from PIL import Image
import os

def convert_webp_to_jpeg(directory):
    for sub_dir in os.listdir(directory):
        sub_dir_path = os.path.join(directory, sub_dir)
        if not os.path.isdir(sub_dir_path):
            continue
        
        for file_name in os.listdir(sub_dir_path):
            file_path = os.path.join(sub_dir_path, file_name)
            if file_name.endswith('.webp'):
                try:
                    img = Image.open(file_path).convert('RGB')  # Convert to RGB for compatibility
                    new_file_path = file_path.rsplit('.', 1)[0] + '.jpg'
                    img.save(new_file_path, 'JPEG')
                    os.remove(file_path)  # Remove the original .webp file
                    print(f"Converted and replaced: {file_path} with {new_file_path}")
                except Exception as e:
                    print(f"Error converting {file_path}: {e}")

# Directories to check and convert
directories = ['dataset/train', 'dataset/validation']

for directory in directories:
    print(f"Converting webp files in {directory}...")
    convert_webp_to_jpeg(directory)


Converting webp files in dataset/train...
Converting webp files in dataset/validation...


In [9]:
import mimetypes

def check_file_types(directory):
    for sub_dir in os.listdir(directory):
        sub_dir_path = os.path.join(directory, sub_dir)
        if not os.path.isdir(sub_dir_path):
            continue
        
        for file_name in os.listdir(sub_dir_path):
            file_path = os.path.join(sub_dir_path, file_name)
            mime_type, _ = mimetypes.guess_type(file_path)
            if mime_type not in ['image/jpeg', 'image/png']:
                print(f"Invalid file type: {file_path} - {mime_type}")
                os.remove(file_path)
                print(f"Removed invalid file: {file_path}")

# Check file types
for directory in ['dataset/train', 'dataset/validation']:
    print(f"Checking file types in {directory}...")
    check_file_types(directory)


Checking file types in dataset/train...
Checking file types in dataset/validation...


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Directories for training and validation data
train_dir = 'dataset/train'
val_dir = 'dataset/validation'

# Image preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Add custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)  # 2 classes: Wearing_ID and No_ID

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Save the model
model.save('id_card_detection_model.h5')
print("Model saved as id_card_detection_model.h5")


Found 239 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


C:\Users\Phani\anaconda3\Lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6715 - loss: 3.7885 - val_accuracy: 0.7000 - val_loss: 5.6012
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7479 - loss: 4.7399 - val_accuracy: 0.9167 - val_loss: 0.5503
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8853 - loss: 2.1170 - val_accuracy: 0.8833 - val_loss: 1.5428
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8475 - loss: 1.4914 - val_accuracy: 0.9000 - val_loss: 0.8233
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8333 - loss: 1.6101 - val_accuracy: 0.9167 - val_loss: 0.5401
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9296 - loss: 0.8174 - val_accuracy: 0.9167 - val_loss: 0.4888
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9386 - loss: 0.3390 - val_accuracy: 0.9333 - val_loss: 0.3494
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9069 - loss: 0.8795 - val_accuracy: 0.9333 - val_loss: 0.3393
Epoch 9/10
8/8 

Model saved as id_card_detection_model.h5


In [11]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('id_card_detection_model.h5')
print("Model loaded successfully.")


Model loaded successfully.


In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('id_card_detection_model.h5')

# Load a pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(
        gray_frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    # Check if any faces are detected
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            # Extract the face region
            face = frame[y:y+h, x:x+w]

            # Preprocess the face for the model
            resized_face = cv2.resize(face, (224, 224))  # Resize to match model input
            normalized_face = resized_face / 255.0  # Normalize pixel values
            input_data = np.expand_dims(normalized_face, axis=0)  # Add batch dimension

            # Make predictions
            predictions = model.predict(input_data, verbose=0)
            class_index = np.argmax(predictions)
            class_label = "Wearing ID" if class_index == 1 else "No ID"

            # Define the rectangle color
            box_color = (0, 255, 0) if class_index == 1 else (0, 0, 255)  # Green for "Wearing ID", Red for "No ID"

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)

            # Display the label near the face
            cv2.putText(frame, class_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

    # Display the frame
    cv2.imshow("ID Card Detection", frame)

    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [2]:
# Count the number of samples in each class
train_counts = train_generator.classes
unique, counts = np.unique(train_counts, return_counts=True)
print("Class distribution:", dict(zip(unique, counts)))


NameError: name 'train_generator' is not defined